In [1]:
import numpy as np
import cv2
import os
import math
import matplotlib.pyplot as plt
import skimage.morphology
from skimage import img_as_ubyte


In [2]:
# Gaps are whether Gaps between words OR Gaps within sub-word
# removing sub-word Gaps
def FilterGaps(G,L):
    
    G = np.array(G)
    L = np.array(L)
    
    IQR = 2
    
    # Filtering Depending On IQR ( Interquartile Range ) Value
    G = G[ L >= IQR ]
    L = L[ L >= IQR ]
    
    Mean = np.mean(L)
    G = G[ L >= Mean ]
    L = L[ L >= Mean ]
    
    
    return G,L

In [3]:
def FindGaps(Line):
    
    G = []
    L = []

    # Vertical Projection Of the text
    VP = np.sum(Line,axis=0)    
    flag = 0
    for i in range(len(VP)):
        
        if VP[i] == 0 :
            if flag == 0:
                # This is a Gap Start 
                G.append(i)
                L.append(1)
                flag = 1
            else:
                # Increment Gap Length
                L[-1] +=1
            
        
        elif flag == 1:
            # End of The Gap
            flag = 0
        
    
    return G,L

In [4]:
def WordSegment(Line):
    
    # G => Array of Gaps, L => Length of each gap
    G,L = FindGaps(Line)
    G,L = FilterGaps(G,L)
    
    # Separete Each Word in a single Image
    Words = [ Line[0:,G[i]+L[i]:G[i+1]] for i in range(len(G)-1) ]    
    
    return Words

In [5]:
def ToBinaryImg(Img):
    ImgGray = cv2.cvtColor(Img, cv2.COLOR_BGR2GRAY)
    BinaryImg = np.array(ImgGray)
    BinaryImg[ ImgGray > 127 ] = 0
    BinaryImg[ ImgGray <= 127 ] = 255
    
    return BinaryImg

In [19]:
def MaxTrans(Line,Baseline):
    
    MaxIdx   = 0
    MaxVal   = 0
    
    for i in range(0,Baseline):
        Trans = 0
        for k in range(1, len(Line[i]) ) :
            if Line[i][k] != Line[i][k-1]:
                Trans +=1
        if Trans > MaxVal:
            MaxVal = Trans
            MaxIdx = i
    
    MTI = []
    for k in range(1, len(Line[MaxIdx]) ):
        if Line[MaxIdx][k] != Line[MaxIdx][k-1]:
            MTI.append(k)
    
    return MaxIdx,np.array(MTI)

In [7]:
def DetectBaseline(Line):
    ThinnedImg = skimage.morphology.thin(Line)
    ThinnedImg = img_as_ubyte(ThinnedImg)
    
    HP = ThinnedImg
    
    Baseline  = np.array( np.sum(ThinnedImg,axis=1) )
    BaseIndex = np.argmax(Baseline) 
    
    return BaseIndex+2
    
    cv2.imshow('Test Image',Line)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imshow('Test Image',ThinnedImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [20]:
def DetectCutPoints(Line):
    pass 
    

In [17]:
Img = cv2.imread('3.png')
dim = Img.shape

BinaryImg = ToBinaryImg(Img)

# Segment Words
Words = WordSegment(BinaryImg)


BaseIndex   = DetectBaseline(BinaryImg)
MaxTransIdx,MTI = MaxTrans(BinaryImg,BaseIndex)   

#Img = cv2.rectangle(Img,(0,BaseIndex),(dim[1],BaseIndex),(0,255,0),1)
Img = cv2.rectangle(Img,(0,MaxTransIdx),(dim[1],MaxTransIdx),(0,255,0),1)

cv2.imshow('BaseLine',Img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('res1.jpg',Img)

#for i in range(len(G)):
#    Img= cv2.rectangle(Img, (G[1]+L[1]-5,0), (G[1]+L[1]-5,dim[1]), (0,255,0), 2)

#for x in Words[::-1]:
#    cv2.imshow('Test Image',x)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
#print(blackAndWhiteImage)


True

In [21]:
#For Code Testing


#Line = np.array([[0,2,3],[0,5,6],[0,7,8]])
#Line = np.sum(Line,axis=1)
#print(Line)

#VP = [ int(np.sum(Line[:,j] ) == 0)  for j in range( len(Line[0]) ) ]

#print(VP)

#x = np.array([1,1,1,0,0,1,1,1,0])

#(~x[:-1] & x[1:]).sum()
#np.count_nonzero(~x[:-1] & x[1:])


#G = np.array([1,2,3,4,5,6,7,8,9])
#G = G[ G < 5 ]
#print(G)
#skimage.morphology.thin